In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from slmsuite.holography.algorithms import Hologram,SpotHologram
from tools.IonChainTools import *
import matplotlib.pyplot as plt
import scipy.constants as con
from math import pi as π

In [ ]:
#loading the slm-200
from slmsuite.hardware.slms.santec import Santec
Santec.info(verbose=True)
#set wavelength to the correct value
slm = Santec(slm_number=1, display_number=2, wav_um=.730, settle_time_s=.3)

In [ ]:
# load the camera
from slmsuite.hardware.cameras.alliedvision import AlliedVision
AlliedVision.info(verbose=True)
#find the camera we want,then choose it
cam = AlliedVision(serial="0245P", verbose=True)
from slmsuite.hardware.cameraslms import FourierSLM
fs = FourierSLM(cam, slm)

In [ ]:
#load vendor-provided wavefront calibraiton file
slm.load_vendor_phase_correction(
 file_path='your path\Wavefront_correction_Data_adjust(730nm).csv', 
 smooth=True
)

In [ ]:
#Fourier calibration
#Remember everytime we move the camera or change the setup we need to do the Fourier calibration again
#When I'm doing the fourier caliration, I put an OD2 and tune the current value to be 24mA for reference. For pratical use, if you find the calibration works not well, please close the camera with demand" cam.close()" and then using the "vimba viewer" to find the best exposure time and current value so that the camera can detect the array dots

cam.set_exposure(100e-6) # set the exposure time for the camera

fs.fourier_calibrate(
    array_shape=[10, 10],           # Size of the calibration grid (Nx, Ny) [knm]
    array_pitch=[18, 18],           # Pitch of the calibration grid (x, y) [knm]
    plot=True
)

In [ ]:
#Remember to save the calibration data after calibration each time and next time you only need to load the data

# fs.save_fourier_calibration()
fs.load_fourier_calibration()

In [ ]:
# functions for caculating the ions' position on the camera
mCa40 = con.atomic_mass*39.9626

def ion_position_act(N,v): #N is the number of the ions, v is the trap frequency
    return calcPositions(N)*lengthScale(v)

def ion_position_cam(N,v,pixel_size,magnititude = 200):
    position = ion_position_act(N,v)*magnititude//pixel_size[0]
    return position

def excite_generator(ions,N,v,pixel_size,pixels):
    '''
    ions: targeted ions;
    N: number of ions;
    v: trap frequency;
    pixel_size: the size of one pixel of camera;
    pixels: the number of pixels of camera
    '''
    position = np.array(ion_position_cam(N,v,pixel_size))
    dy = pixels[0]//8 
    all = range(N)
    pos = position[ions]
    position_cam = np.column_stack((pos+pixels[1]//2,np.full(pos.shape, pixels[0]//2+dy)))
    position_cam = position_cam.T
    holo = SpotHologram(shape=(2048,2048), 
            spot_vectors=position_cam, 
            basis="ij", 
            cameraslm=fs)
    holo.optimize('WGS-Kim', feedback='computational_spot', stat_groups=['computational_spot'], maxiter=20)
    holo.optimize('WGS-Kim', feedback='experimental_spot', stat_groups=['computational_spot'], maxiter=30) #use camera feedback
    
    phase = holo.extract_phase()
    
    return phase


In [ ]:
#the camera we are using has pixels:1944*2592, with pixel_size of 2.2um*2.2um

N = 6  # The number of the ions

v = 2*π*1e6   # trap frequency  
                 
pixel_size = np.array([2.2e-6,2.2e-6]) 
pixels = np.array([1944,2592])

ions = np.array([0,1,2,3,4,5]) #Here you determine the ions you want to excite.

phase  = excite_generator(ions,N,v,pixel_size,pixels)
slm.write(phase) #write phase to the slm

In [ ]:
#plot the image from the camera with circles enclosed the spots
cam.set_exposure(100e-6)
cam.flush()
img = cam.get_image()
position = ion_position_cam(N,v,pixel_size)
dy = pixels[0]//8
position_cam = np.column_stack((position+pixels[1]//2,np.full(position.shape, pixels[0]//2+dy)))
for i, pos in enumerate(position_cam):
    cv2.circle(img, (int(pos[0]), int(pos[1])), 60, (255, 0, 0), 2)
plt.imshow(img)

In [ ]:
#calculate the intensity on different ions
cal = []
cal.append(np.sum(img[1944//2-500-60:1944//2-500+60,2592//2-60:2592//2+60])/255.0) #a reference data for background intensity
for i, pos in enumerate(position_cam):
    cal.append(np.sum(img[int(pos[1])-60:int(pos[1])+60,int(pos[0])-60:int(pos[0])+60])/255.0)

In [ ]:
#plot the intensity at different ions
plt.scatter(range(7),cal, c='blue', marker='o', label='data')
plt.xlabel('ions')
plt.ylabel('intensity')
# plt.ylim(1250, 2250)
# plt.yticks(np.arange(1250,2250,125))
plt.legend()
plt.show()

In [ ]:
#calculate the fidelity
from numpy import sum, sqrt, max
import numpy as np
intensity = cal[1:]
all = range(N)
undesire = [ i for i in all if i not in ions]
intensity = np.array(intensity)
# background = result[0], Here is a little different from using the blaze grating phase, I recommend you use the background data calculated by using the blaze grating phase due to MRAF's property
# The below is the data in my test as a reference
intensity_non = intensity[undesire]
intensity_non = np.append(intensity_non,cal[0])
background = np.min(intensity_non)

inten = np.array([ i- background for i in intensity])
inten[inten < 0] = 0

Omega_desire = sqrt(inten[ions])
Omega_undesire = sqrt(inten[undesire])
F1 = sum(Omega_desire)/(len(ions)*max(Omega_desire))
F2 = 1-sum(Omega_undesire)/(len(undesire)*max(Omega_desire))
F = F1*F2

print(F1)
print(F2)
print(F)